<a href="https://colab.research.google.com/github/jaydebsarker/AuthorProfilingAbuseDetection/blob/master/gy3312ML1project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#CSC5825
#Fall2020
#Name: Jaydeb Sarker
#ID: gy3312
#credit card fraud detection

In [2]:
#import classifiers and others
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.utils import resample 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import collections

from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from imblearn.metrics import classification_report_imbalanced

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

In [3]:
def read_data():
    data=pd.read_csv('creditcard.csv')
    return data

data=read_data() 



In [5]:
from google.colab import files
uploaded = files.upload()

Saving creditcard.csv to creditcard.csv


In [20]:
import io
data= pd.read_csv(io.BytesIO(uploaded['creditcard.csv'] ))

In [21]:
print(data["Class"].value_counts())

0    284315
1       492
Name: Class, dtype: int64


In [22]:
print('The data without fraud: ',round(data["Class"].value_counts()[0]/len(data)*100,3))
print('The data with fraud:     ',round(data["Class"].value_counts()[1]/len(data)*100,3))

The data without fraud:  99.827
The data with fraud:      0.173


In [23]:
##apply preprocessing
columns= data[['Time', 'Amount']]
pca = PCA()
pca.fit(columns)
X_PCA = pca.transform(columns)

data['PCATime']=X_PCA[:,0]
data['PCA amount']=X_PCA[:,1]

data.drop(['Time', 'Amount'], axis=1, inplace=True)

data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Class', 'PCATime',
       'PCA amount'],
      dtype='object')

In [24]:
# (7:3) split for supervised learning
X_train, X_test= train_test_split(data, test_size=0.3, random_state=7)

In [25]:
print(X_train.shape)

(199364, 31)


In [26]:
print(X_test['Class'].value_counts())

#test data without upsampling
X_test_t= X_test.drop(["Class"], axis=1)
Y_test= X_test["Class"]

0    85288
1      155
Name: Class, dtype: int64


In [27]:
#apply upsampling to the trained data
maj_class=X_train[X_train["Class"]==0]
min_class=X_train[X_train["Class"]==1]


resamp_minclass=resample(min_class,n_samples=199027,replace=True,random_state=42)
print(resamp_minclass.shape)

Xtrain_new= pd.concat([maj_class,resamp_minclass])
Xtrain_new["Class"].value_counts()

(199027, 31)


1    199027
0    199027
Name: Class, dtype: int64

In [28]:
upsampling_X= Xtrain_new.drop(["Class"], axis=1)
upsampling_y= Xtrain_new["Class"]

In [29]:
print(upsampling_X.shape)

(398054, 30)


In [35]:
##apply supervised models
from sklearn.naive_bayes import GaussianNB
upsampling_pipe = []

upsampling_pipe.append(('upsamp_LR', Pipeline([('samp_LR',LogisticRegression(max_iter=1000))])))

upsampling_pipe.append(('upsamp_DCT', Pipeline([('DCT',DecisionTreeClassifier())])))

upsampling_pipe.append(('upsamp_NB', Pipeline([('NB',GaussianNB())])))



In [36]:
upsampling_y.value_counts()

1    199027
0    199027
Name: Class, dtype: int64

In [37]:
#10 fold cross validation after upsampling
#10 fold cross validation after upsampling

upsamp_results = []
names = []
for name, model in upsampling_pipe:
    kfold = KFold(n_splits=10, random_state=None, shuffle=False)
    ups_cv_results = cross_val_score(model, upsampling_X, upsampling_y, cv=kfold)
    upsamp_results.append(ups_cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, ups_cv_results.mean(), ups_cv_results.std())
    print(msg)

upsamp_LR: 0.930172 (0.035705)
upsamp_DCT: 0.999797 (0.000234)
upsamp_NB: 0.862879 (0.127681)


In [41]:
##apply logistic regression to the train data

LR=LogisticRegression(max_iter=1000)
                
kF= KFold(n_splits=10, random_state=None, shuffle=False)
LR.fit(upsampling_X, upsampling_y)
LR_predictions=LR.predict(X_test_t)

score=roc_auc_score(Y_test, LR_predictions)
print("Score of LR is: ", score)

LR_predictions
print(accuracy_score(Y_test,LR_predictions))

from sklearn.metrics import classification_report
print(classification_report(Y_test, LR_predictions))

Score of LR is:  0.9451377268972528
0.9740411736479291
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     85288
           1       0.06      0.92      0.11       155

    accuracy                           0.97     85443
   macro avg       0.53      0.95      0.55     85443
weighted avg       1.00      0.97      0.99     85443



In [43]:
##Apply Decision Tree for train and test
DT=DecisionTreeClassifier()
                
kF= KFold(n_splits=10, random_state=None, shuffle=False)
DT.fit(upsampling_X, upsampling_y)
DT_predictions=DT.predict(X_test_t)

score=roc_auc_score(Y_test, DT_predictions)
print("Score of DT is: ", score)

 
print(accuracy_score(Y_test, DT_predictions))

from sklearn.metrics import classification_report
print(classification_report(Y_test, DT_predictions))




Score of DT is:  0.8868564121262
0.9991105181231933
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85288
           1       0.75      0.77      0.76       155

    accuracy                           1.00     85443
   macro avg       0.87      0.89      0.88     85443
weighted avg       1.00      1.00      1.00     85443



In [40]:
##apply Naive Bayes to the train data and test it

NB=GaussianNB() 
                
kF= KFold(n_splits=10, random_state=None, shuffle=False)
NB.fit(upsampling_X, upsampling_y)
NB_predictions=NB.predict(X_test_t)

score=roc_auc_score(Y_test, NB_predictions)
print("Score of NB is: ", score)

 
print(accuracy_score(Y_test, NB_predictions))

from sklearn.metrics import classification_report
print(classification_report(Y_test, NB_predictions))

Score of NB is:  0.8922952516104826
0.9906838477113398
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     85288
           1       0.14      0.79      0.24       155

    accuracy                           0.99     85443
   macro avg       0.57      0.89      0.62     85443
weighted avg       1.00      0.99      0.99     85443



In [ ]:
SVM=SVC()
kF= KFold(n_splits=10, random_state=None, shuffle=False)
SVM.fit(upsampling_X, upsampling_y)
SVM_predictions=SVM.predict(X_test_t)

score=roc_auc_score(Y_test, SVM_predictions)
print("Score of SVM is: ", score)

 
print(accuracy_score(Y_test, SVM_predictions))

from sklearn.metrics import classification_report
print(classification_report(Y_test, SVM_predictions))

In [ ]:
## Apply Unsupervised Learning Method Isolation Forest

#Feature Selection using Z-test
''' 
Valid transactions as our population
Fraud transactions as sample
Two tailed Z-test
Level of significance 0.01
Corresponding critical value is 2.58
'''



In [ ]:
def ztest(feature):
    
    mean = normal[feature].mean()
    std = fraud[feature].std()
    zScore = (fraud[feature].mean() - mean) / (std/np.sqrt(sample_size))
    
    return zScore

In [ ]:
columns= data.drop('Class', axis=1).columns
normal= data[data.Class==0]
fraud= data[data.Class==1]
sample_size=len(fraud)
significant_features=[]
critical_value=2.58


for i in columns:
    
    z_vavlue=ztest(i)
    
    if( abs(z_vavlue) >= critical_value):    
        print(i," is statistically significant") #Reject Null hypothesis. i.e. H0
        significant_features.append(i)

In [ ]:
significant_features.append('Class')
data= data[significant_features]
print(data.shape)

In [ ]:
##verify fraud and non-fraud
Fraud = data[data['Class'] == 1]
Valid = data[data['Class'] == 0]

outlier_fraction = len(Fraud)/float(len(Valid))
print(outlier_fraction)

print('Fraud Cases: {}'.format(len(data[data['Class'] == 1])))
print('Valid Transactions: {}'.format(len(data[data['Class'] == 0])))

In [ ]:
columns = data.columns.tolist()

# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Class"]]

# Store the variable we'll be predicting on
target = "Class"

X = data[columns]
Y = data[target]

# Print shapes
print(X.shape)
print(Y.shape)

In [ ]:
## Applying Isolation Forest
state= 42
IF = IsolationForest(n_estimators = 1050, max_features = 1.0, max_samples=1.0, 
                        behaviour="new", bootstrap=False, random_state=22,
                        contamination = outlier_fraction)
#ISF = IsolationForest(random_state=state)
IF.fit(X)

scores_prediction = IF.decision_function(X)
y_pred = IF.predict(X)


#Reshape the prediction values 
y_pred[y_pred == 1] = 0
y_pred[y_pred == -1] = 1
n_errors = (y_pred != Y).sum()

from sklearn.metrics import classification_report,accuracy_score
print("Accuracy Score :")
print(accuracy_score(Y,y_pred))
print("Classification Report :")
print(classification_report(Y,y_pred))